In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd "drive/My Drive/VQA-master"

/content/drive/My Drive/VQA-master


In [0]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image
from torch.utils import data
import numpy as np
import torch
import torch.optim as optim

In [0]:
use_image_features=True
batch_size = 1000
print_every = 10

input_options = {1: "resnet 18 image features, question", 2: "resnet 192 image features, question", 3: "new resnet 18 features", 4: "new resnet 152 features"}
model_input = input_options[3]

answer_type_options = {1: "yesno", 2: "number", 3: "other"}
answer_type = answer_type_options[3]

if answer_type == "yesno":
  lstm_hidden_size = 64
  lstm_num_layers = 1  
else:
  lstm_hidden_size = 256
  lstm_num_layers = 1

fusion_type_options = {1: "concatenation", 2: "pointwise_mul", 3: "try"}
fusion_type = fusion_type_options[3]

model_type_options = {1: "simplecnnlstm", 2: "fusion", 3: "transformer"}
model_type = model_type_options[3]

#works only for fusion model
dropout_amount = 0.05

Top100 = False

#### Load data

In [5]:
new_mapping = {}
train_questions = np.load("train_questions.npy")
val_questions = np.load("val_questions.npy")
cnt = 0
for i in range(train_questions.shape[0]):
  for j in range(train_questions.shape[1]):
    v = train_questions[i][j]
    if v not in new_mapping:
      new_mapping[v] = cnt
      cnt += 1
for i in range(val_questions.shape[0]):
  for j in range(val_questions.shape[1]):
    v = val_questions[i][j]
    if v not in new_mapping:
      new_mapping[v] = cnt
      cnt += 1
print(cnt)

2692


In [6]:
new_mapping2 = {}
train_questions = np.load("train_questions_"+answer_type+".npy")
val_questions = np.load("val_questions_"+answer_type+".npy")
cnt2 = 0
for i in range(train_questions.shape[0]):
  for j in range(train_questions.shape[1]):
    v = train_questions[i][j]
    if v not in new_mapping2:
      new_mapping2[v] = cnt2
      cnt2 += 1
for i in range(val_questions.shape[0]):
  for j in range(val_questions.shape[1]):
    v = val_questions[i][j]
    if v not in new_mapping2:
      new_mapping2[v] = cnt2
      cnt2 += 1
print(cnt2)

2133


In [0]:
from torch.utils import data

if model_input == "resnet 18 image features, question":
  # train_answers = np.load("train_answers_"+answer_type+".npy")
  if Top100 == True:
    train_indices = np.load("top100_train_indices.npy")
    val_indices = np.load("top100_val_indices.npy")
    
  train_answers = np.load("train_answers.npy")
  if Top100 == True:
    
    train_answers = train_answers[train_indices]
    answer_dict = {}
    cnt = 0
    print(train_answers.shape)
    for i in range(train_answers.shape[0]):
      v = train_answers[i][0]
      if v not in answer_dict:
        answer_dict[v] = cnt
        cnt += 1
    print(cnt)

    print(answer_dict)
    for i in range(train_answers.shape[0]):
      train_answers[i,0] = answer_dict[train_answers[i,0]]
    # train_answers = answer_dict[train_answers[:,0]]
    # print(train_answers.shape)

  tensor_train_answers = torch.Tensor(train_answers).flatten()
  tensor_train_answers = tensor_train_answers.type(torch.long)

  # val_answers = np.load("val_answers_"+answer_type+".npy")
  val_answers = np.load("val_answers.npy")
  if Top100 == True:
    val_answers = val_answers[val_indices]
    for i in range(val_answers.shape[0]):
      val_answers[i,0] = answer_dict[val_answers[i,0]]
  tensor_val_answers = torch.Tensor(val_answers).flatten()
  tensor_val_answers = tensor_val_answers.type(torch.long)

  
elif model_input == "resnet 192 image features, question":
  train_answers = np.load("train_answers_"+answer_type+".npy")
  # train_answers = np.load("train_answers.npy")
  tensor_train_answers = torch.Tensor(train_answers).flatten()
  tensor_train_answers = tensor_train_answers.type(torch.long)

  val_answers = np.load("val_answers_"+answer_type+".npy")
  # val_answers = np.load("val_answers.npy")
  tensor_val_answers = torch.Tensor(val_answers).flatten()
  tensor_val_answers = tensor_val_answers.type(torch.long)

  # train_indices = np.load("top100_train_indices.npy")
  # val_indices = np.load("top100_val_indices.npy")
elif model_input == "new resnet 18 features":
  train_answers = np.load("filtered_train_answers.npy")
  # train_answers = np.load("train_answers.npy")
  tensor_train_answers = torch.Tensor(train_answers).flatten()
  tensor_train_answers = tensor_train_answers.type(torch.long)

  val_answers = np.load("filtered_val_answers.npy")
  # val_answers = np.load("val_answers.npy")
  tensor_val_answers = torch.Tensor(val_answers).flatten()
  tensor_val_answers = tensor_val_answers.type(torch.long)

  # train_indices = np.load("top100_train_indices.npy")
  # val_indices = np.load("top100_val_indices.npy")


if model_input == "resnet 18 image features, question":
  # train_images = np.load("train_image_features_"+answer_type+".npy")
  # train_questions = np.load("train_questions_"+answer_type+".npy")

  # val_images = np.load("val_image_features_"+answer_type+".npy")
  # val_questions = np.load("val_questions_"+answer_type+".npy")

  train_images = np.load("train_image_features.npy")
  train_questions = np.load("train_questions.npy")
  for i in range(train_questions.shape[0]):
    for j in range(train_questions.shape[1]):
      v = train_questions[i,j]
      train_questions[i,j] = new_mapping[v]
  # train_questions = train_questions[:,:20]

  if Top100 == True:
    train_images = train_images[train_indices]
    train_questions = train_questions[train_indices]

  val_images = np.load("val_image_features.npy")
  val_questions = np.load("val_questions.npy")
  for i in range(val_questions.shape[0]):
    for j in range(val_questions.shape[1]):
      v = val_questions[i,j]
      val_questions[i,j] = new_mapping[v]

  # val_questions = val_questions[:,:20]

  if Top100 == True:
    val_images = val_images[val_indices]
    val_questions = val_questions[val_indices]

  tensor_train_images = torch.Tensor(train_images)
  tensor_train_questions = torch.Tensor(train_questions)

  tensor_val_images = torch.Tensor(val_images)
  tensor_val_questions = torch.Tensor(val_questions)

  trainset = data.TensorDataset(tensor_train_images,tensor_train_questions,tensor_train_answers)
  valset = data.TensorDataset(tensor_val_images,tensor_val_questions,tensor_val_answers)

  num_workers = 2

elif model_input == "resnet 192 image features, question":
  train_images = np.load("train_image_features_"+answer_type+".npy")
  train_questions = np.load("train_questions_"+answer_type+".npy")

  val_images = np.load("val_image_features_"+answer_type+".npy")
  val_questions = np.load("val_questions_"+answer_type+".npy")
  
  for i in range(train_questions.shape[0]):
    for j in range(train_questions.shape[1]):
      v = train_questions[i,j]
      train_questions[i,j] = new_mapping2[v]


  for i in range(val_questions.shape[0]):
    for j in range(val_questions.shape[1]):
      v = val_questions[i,j]
      val_questions[i,j] = new_mapping2[v]

  tensor_train_images = torch.Tensor(train_images)
  tensor_train_questions = torch.Tensor(train_questions)

  tensor_val_images = torch.Tensor(val_images)
  tensor_val_questions = torch.Tensor(val_questions)

  trainset = data.TensorDataset(tensor_train_images,tensor_train_questions,tensor_train_answers)
  valset = data.TensorDataset(tensor_val_images,tensor_val_questions,tensor_val_answers)

  num_workers = 2

elif model_input == "new resnet 18 features":
  train_images = np.load("filtered_train_image_features_18.npy")
  train_questions = np.load("filtered_train_questions.npy")

  val_images = np.load("filtered_val_image_features_18.npy")
  val_questions = np.load("filtered_val_questions.npy")
  
  tensor_train_images = torch.Tensor(train_images)
  tensor_train_questions = torch.Tensor(train_questions)

  tensor_val_images = torch.Tensor(val_images)
  tensor_val_questions = torch.Tensor(val_questions)

  trainset = data.TensorDataset(tensor_train_images,tensor_train_questions,tensor_train_answers)
  valset = data.TensorDataset(tensor_val_images,tensor_val_questions,tensor_val_answers)

  num_workers = 2



trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True, num_workers=num_workers)


In [8]:
print(train_images.shape)
print(val_images.shape)

(13000, 512)
(2600, 512)


In [9]:
print(train_questions.shape)
print(val_questions.shape)

(13000, 25)
(2600, 25)


#### Choose model

In [0]:
if model_type == "simplecnnlstm":
  import models.Simple_CNN_LSTM_Model as models
  # import models.trial_model as trial_model
elif model_type == "fusion":
  import models.fusion_model as fusion_model
elif model_type == "transformer":
  import models.transformer_model as transformer_model

In [11]:
print(lstm_hidden_size)

256


In [0]:
weights_matrix = np.load("weights_matrix.npy")
weights_matrix = torch.tensor(weights_matrix)
if model_type == "simplecnnlstm":
  model = models.SimpleCNNLSTM(weights_matrix, lstm_hidden_size, lstm_num_layers, model_input, answer_type, fusion_type, Top100=Top100)
elif model_type == "fusion":
  if answer_type == "yesno":
    model = fusion_model.FusionModel(weights_matrix, lstm_hidden_size, lstm_num_layers, model_input, answer_type, fusion_type, shared_size=32, dropout_amount=dropout_amount, Top100=Top100)
  else:
    model = fusion_model.FusionModel(weights_matrix, lstm_hidden_size, lstm_num_layers, model_input, answer_type, fusion_type, shared_size=256, dropout_amount=dropout_amount, Top100=Top100)
elif model_type == "transformer":
  if answer_type == "yesno":
    model = transformer_model.FusionModel(weights_matrix, lstm_hidden_size, lstm_num_layers, model_input, answer_type, fusion_type, shared_size=32, dropout_amount=dropout_amount, Top100=Top100)
  else:
    model = transformer_model.FusionModel(weights_matrix, lstm_hidden_size, lstm_num_layers, model_input, answer_type, fusion_type, shared_size=256, dropout_amount=dropout_amount, Top100=Top100)

In [13]:
print(model)

FusionModel(
  (transformer_model): TransformerModel(
    (pos_encoder): PositionalEncoding(
      (dropout): Dropout(p=0.05, inplace=False)
    )
    (transformer_encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (linear1): Linear(in_features=512, out_features=512, bias=True)
          (dropout): Dropout(p=0.05, inplace=False)
          (linear2): Linear(in_features=512, out_features=512, bias=True)
          (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.05, inplace=False)
          (dropout2): Dropout(p=0.05, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): Linear(in_features=512, out_features=512,

#### Train

In [0]:
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
lr_reducer = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=np.sqrt(0.1),
                                                                        cooldown=0,
                                                                        patience=3,
                                                                        min_lr=0.5e-9)

In [0]:
def get_accuracy(predictions, labels):
  predictions = torch.max(predictions, axis=1)[1]
  ab = torch.abs(predictions-labels)
  ab = ab.detach().numpy()
  mn = np.minimum(ab, 1)
  eq = 1-mn
  correct = np.sum(eq)
  total = eq.shape[0]
  return correct, total

In [0]:
def focal_loss(y_pred, y_true, gamma=2., alpha=4.):

    gamma = float(gamma)
    alpha = float(alpha)

    def focal_loss_fixed(y_true, y_pred):
        """Focal loss for multi-classification
        FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
        Notice: y_pred is probability after softmax
        gradient is d(Fl)/d(p_t) not d(Fl)/d(x) as described in paper
        d(Fl)/d(p_t) * [p_t(1-p_t)] = d(Fl)/d(x)
        Focal Loss for Dense Object Detection
        https://arxiv.org/abs/1708.02002

        Arguments:
            y_true {tensor} -- ground truth labels, shape of [batch_size, num_cls]
            y_pred {tensor} -- model's output, shape of [batch_size, num_cls]

        Keyword Arguments:
            gamma {float} -- (default: {2.0})
            alpha {float} -- (default: {4.0})

        Returns:
            [tensor] -- loss.
        """
        # y_pred = y_pred.detach().numpy()
        # y_pred = np.array(y_pred, dtype=np.int32)
        # print(y_true.shape)
        # print(y_pred.shape)
        # y_pred_onehot = np.zeros(shape=[y_pred.shape[0], y_true.size(1)])
        # y_pred_onehot[:,y_pred] = 1
        # if answer_type != "yesno":
        y_true_onehot = torch.FloatTensor(y_pred.size())
        y_true_onehot.zero_()
        y_true_onehot.scatter_(1, y_true.type(torch.long).view(y_true.size(0),1), 1.0)
        y_true = y_true_onehot

        epsilon = 1.e-9
        # y_true = tf.convert_to_tensor(y_true, tf.float32)
        # y_pred = tf.convert_to_tensor(y_pred, tf.float32)
        y_pred = y_pred.type(torch.float32)
        y_true = y_true.type(torch.float32)

        # print(torch.max(y_true[:10], axis=1))
        # print(torch.max(y_pred[:10], axis=1))

        model_out = torch.add(y_pred, epsilon)
        # print("model_out: ",model_out[:1])
        # model_out = tf.add(y_pred, epsilon)
        ce = torch.mul(y_true, torch.log(model_out))
        # print("ce: ",ce[:1])
        # ce = tf.multiply(y_true, -tf.log(model_out))
        # print("1-model_out: ",(1-model_out)[:1])
        # print("gamma: ",gamma)
        # print("pow: ",torch.pow((1-model_out), gamma)[:1])
        weight = torch.mul(y_true, torch.pow((1-model_out), gamma))
        # print("weight: ",weight[:1])
        # weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
        fl = -1*torch.mul(torch.mul(alpha, weight), ce)
        # print("fl: ",fl[:1])
        # fl = tf.multiply(alpha, tf.multiply(weight, ce))
        reduced_fl = torch.max(fl, axis=1)[0]
        # print("reduced_fl: ",reduced_fl[:1])
        # reduced_fl = tf.reduce_max(fl, axis=1)
        return torch.mean(reduced_fl)
        # return tf.reduce_mean(reduced_fl)
    loss = focal_loss_fixed(y_true, y_pred)
    # print(loss)
    return loss

alpha = 4.
gamma = 2.

In [0]:
for epoch in range(500):  # loop over the dataset multiple times

    running_loss = 0.0
    correct = 0
    total = 0
    total_loss = 0.0
    model.train()
    num_batches = 0

    for i, data in enumerate(trainloader, 0):
    # for i, data in enumerate(valloader, 0):
        # if i % print_every == 0 and i>0:
          # print("Train batch ",i+1)

        optimizer.zero_grad()

        if model_input == "resnet 18 image features, question" or model_input == "resnet 192 image features, question" or model_input == "new resnet 18 features":
          image_features, questions, labels = data
          # print(labels.size())
          image_features = image_features.type(torch.float32)
          labels = labels.flatten().type(torch.long)
          labels = labels.view(-1)

          outputs = model(image_features, questions)
          # print("out done")

        
        batch_correct, batch_total = get_accuracy(outputs, labels)
        # print("accuracy done")
        correct += batch_correct
        total += batch_total
        # loss = focal_loss(outputs, labels, alpha=alpha, gamma=gamma)
        loss = criterion(outputs, labels)
        # print("loss done")
        loss.backward()
        # print("backprop done")
        optimizer.step()
        # print("optim done")

        running_loss += loss.item()
        total_loss += running_loss
        num_batches += 1
        running_loss = 0.0

    val_correct = 0
    val_total = 0
    val_batches = 0
    val_total_loss = 0.0
    model.eval()
    with torch.no_grad():
      # for i, data in enumerate(trainloader, 0):
      for i, data in enumerate(valloader, 0):
          # if i % print_every == 0 and i>0:
            # print("Val batch ",i+1)

          if model_input == "resnet 18 image features, question" or model_input == "resnet 192 image features, question" or model_input == "new resnet 18 features":
            image_features, questions, labels = data
            image_features = image_features.type(torch.float32)
            labels = labels.flatten().type(torch.long)
            labels = labels.view(-1)

            outputs = model(image_features, questions)

          batch_correct, batch_total = get_accuracy(outputs, labels)
          val_correct += batch_correct
          val_total += batch_total
          val_batches += 1

          # val_loss = focal_loss(outputs, labels, alpha=alpha, gamma=gamma)
          val_loss = criterion(outputs, labels)
          val_total_loss += val_loss.detach().numpy()
          lr_reducer.step(val_loss)

          
    print("Epoch: ",epoch+1," Loss: ",total_loss/num_batches," Train-Accuracy: ", correct/total," Val Loss: ",val_total_loss/val_batches," Val-Accuracy: ",val_correct/val_total)
    

print('Finished Training')

Epoch:  1  Loss:  2.478324321600107  Train-Accuracy:  0.20553846153846153  Val Loss:  2.494877338409424  Val-Accuracy:  0.26153846153846155
Epoch:  2  Loss:  2.3846846177027774  Train-Accuracy:  0.2980769230769231  Val Loss:  2.459228833516439  Val-Accuracy:  0.2773076923076923
Epoch:  3  Loss:  2.3464840742257924  Train-Accuracy:  0.3396923076923077  Val Loss:  2.456593672434489  Val-Accuracy:  0.26346153846153847
Epoch:  4  Loss:  2.3321154484382043  Train-Accuracy:  0.35107692307692306  Val Loss:  2.4404305617014566  Val-Accuracy:  0.2796153846153846
Epoch:  5  Loss:  2.298803604566134  Train-Accuracy:  0.3859230769230769  Val Loss:  2.4459386666615806  Val-Accuracy:  0.29115384615384615
Epoch:  6  Loss:  2.2647545337677  Train-Accuracy:  0.42315384615384616  Val Loss:  2.439512252807617  Val-Accuracy:  0.30153846153846153
Epoch:  7  Loss:  2.239628003193782  Train-Accuracy:  0.44969230769230767  Val Loss:  2.4371068477630615  Val-Accuracy:  0.3
Epoch:  8  Loss:  2.218923642085149  

In [0]:
cnt_dict = {}

for i in range(len(train_answers)):
  v = train_answers[i][0]
  if v not in cnt_dict:
    cnt_dict[v] = 1
  else:
    cnt_dict[v] = cnt_dict[v] + 1
print(cnt_dict)

cnt_list = []
for key in cnt_dict.keys():
  v = cnt_dict[key]
  if v > 50:
    cnt_list.append((key,v))

print(len(cnt_list))